In [1]:
import sys
sys.path.append("..")

from modal.amodal import Amodal
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import random,pickle
import cv2,json,os
import numpy as np
from skimage.measure import regionprops
from pycocotools import mask as maskUtils

In [2]:
def createAmodalRegion(ann, id):
    region = {}
    region['id'] = id #used for gt/dt matching
    region['segmentation'] = ann['segmentation']
    region['score'] = ann['score']
    region['isStuff'] = 0  # default things
    if 'foreground_ness' in ann:
        region['foreground_ness'] = ann['foreground_ness']
    if 'invisibleMask' in ann:
        region['invisible_mask'] = ann['invisibleMask']
    if 'amodalMask' in ann:
        region['amodal_mask'] = ann['amodalMask']
    return region

def createAmodalAnn(image_id, ann_id):
    ann = {}
    ann['id'] = ann_id
    ann['category_id'] = 1 # fake label
    ann['image_id'] = image_id
    ann['regions']  =[]
    return ann

def filterDtFile(resFiles, amodalGtImgIds):
    amodalDt = {}
    id = 0
    ann_id = 0
    for i, file in enumerate(resFiles):
        print("processing json %d in total %d" %(i+1, len(resFiles)))
        anns = json.load(open(file))
        for ann in anns:
            image_id = ann['image_id']
            if image_id in amodalGtImgIds:
                id = id + 1
                if image_id not in amodalDt:
                    amodalDt[image_id] = createAmodalAnn(image_id, ann_id)
                    ann_id = ann_id + 1
                region = createAmodalRegion(ann, id)
                amodalDt[image_id]['regions'].append(region)
    res = []
    for image_id, ann in amodalDt.items():
        res.append(ann)
    return res

def refine_detections(rois, probs, deltas, window, config):
    """Refine classified proposals and filter overlaps and return final
    detections.

    Inputs:
        rois: [N, (y1, x1, y2, x2)] in normalized coordinates
        probs: [N, num_classes]. Class probabilities.
        deltas: [N, num_classes, (dy, dx, log(dh), log(dw))]. Class-specific
                bounding box deltas.
        window: (y1, x1, y2, x2) in image coordinates. The part of the image
            that contains the image excluding the padding.

    Returns detections shaped: [N, (y1, x1, y2, x2, class_id, score)]
    """

    # Class IDs per ROI
    _, class_ids = torch.max(probs, dim=1)

    # Class probability of the top class of each ROI
    # Class-specific bounding box deltas
    idx = torch.arange(class_ids.size()[0]).long()
#     if config.GPU_COUNT:
#         idx = idx.cuda()
    class_scores = probs[idx, class_ids.data]
    deltas_specific = deltas[idx, class_ids.data]

#    refined_rois = coordinate_convert(rois,deltas_specific,config,config.GPU_COUNT)

    # Clip boxes to image window
    refined_rois = clip_to_window(window, refined_rois)

    # Round and cast to int since we're deadling with pixels now
    refined_rois = torch.round(refined_rois)

    # TODO: Filter out boxes with zero area

    # Filter out background boxes

    keep_bool = class_ids>0
    if config.USE_NMS:
        # Filter out low confidence boxes
        if config.DETECTION_MIN_CONFIDENCE:
            keep_bool = keep_bool & (class_scores >= config.DETECTION_MIN_CONFIDENCE)

        keep = torch.nonzero(keep_bool)
        if len(keep)==0:
            return [],[]

        keep = keep[:,0]

        # Apply per-class NMS
        pre_nms_class_ids = class_ids[keep.data]
        pre_nms_scores = class_scores[keep.data]
        pre_nms_rois = refined_rois[keep.data]

        for i, class_id in enumerate(unique1d(pre_nms_class_ids)):
            # Pick detections of this class
            ixs = torch.nonzero(pre_nms_class_ids == class_id)[:,0]

            # Sort
            ix_rois = pre_nms_rois[ixs.data]
            ix_scores = pre_nms_scores[ixs]
            ix_scores, order = ix_scores.sort(descending=True)
            ix_rois = ix_rois[order.data,:]

            class_keep = nms(torch.cat((ix_rois, ix_scores.unsqueeze(1)), dim=1).data, config.DETECTION_NMS_THRESHOLD)

            # Map indicies
            class_keep = keep[ixs[order[class_keep].data].data]

            if i==0:
                nms_keep = class_keep
            else:
                nms_keep = unique1d(torch.cat((nms_keep, class_keep)))
        keep = intersect1d(keep, nms_keep)
    else:

        keep = torch.nonzero(keep_bool).view((-1))
        if keep[-1] > 10:

            ix_scores, order  = class_scores[keep.data].sort(descending=True)
            keep = keep[order[:3]]



    # Keep top detections
    roi_count = config.DETECTION_MAX_INSTANCES
    print(len(keep.data))
    if len(keep.data)>0:
        top_ids = class_scores[keep.data].sort(descending=True)[1][:]
        keep = keep[top_ids.data]

        # Arrange output as [N, (y1, x1, y2, x2, class_id, score)]
        # Coordinates are in image domain.
        result = torch.cat((refined_rois[keep.data],
                            class_ids[keep.data].unsqueeze(1).float(),
                            class_scores[keep.data].unsqueeze(1)), dim=1)
    else:
        return [],[]

    return result,refined_rois

In [3]:
#################################################################
######################  COCOA dataset  ############################

In [4]:
def mask_to_bbox(mask):
    props = regionprops(mask)

    bbox,maxarea = [],0
    for prop in props:
        y1,x1,y2,x2 = prop.bbox
        areas = (y2-y1)*(x2-x1)
        if areas>maxarea:
            bbox = prop.bbox
    return bbox

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

In [8]:
ann_file2 = '../datasets/coco_amodal/annotations/COCO_amodal_val2014.json'
amodal = Amodal(ann_file2)
amodalGtImgIds = amodal.getImgIds()
img_info = amodal.dataset['images']

loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [13]:
##########################################
##########  AmodalMask[39] results #######
ann_file = '../results/AmodalMask[39]/amodal-props-1-500.json'
anns = json.load(open(ann_file))
dt = filterDtFile([ann_file], amodalGtImgIds)
dt = sorted(dt, key=lambda reg : reg['image_id'])

startInd = 0 
## please modify this value if change amodal annotations
## amodal-props-501-1000: 501   
## amodal-props-1001-1323:1001

targetFolder = '../results/AmodalMask[39]/'  ## path to target folder
targetFolder = os.path.abspath(targetFolder)
if not os.path.exists(targetFolder):
    os.mkdir(targetFolder)


for i in range(startInd,len(img_info)):
    dt_bboxes = []
    for region in dt[i]['regions']:
        mask = maskUtils.decode(region['segmentation'])
        # TO DO get bounding box
        bbox = mask_to_bbox(mask)
        dt_bboxes.append(bbox)
        
    imgIds = dt[i]['image_id']
    amodal_bboxes = []
    for region in amodal.dataset['annotations'][i]['regions']:
        amodal_mask,vis_mask = amodal.getAnnMask2(region,img_info[i]['width'],img_info[i]['height'])
        bbox = mask_to_bbox(amodal_mask)
        amodal_bboxes.append(bbox)
        
    
    dt_bboxes,amodal_bboxes = np.array(dt_bboxes),np.array(amodal_bboxes)
    
    if dt_bboxes.shape!=(1000,4):
        continue

        
    ## TO DO
    # Input: bboxes 1000 ,amodal_mask object N
    # output visualize result 
    final_index = []
    for j in range(amodal_bboxes.shape[0]):
        dis = np.sum(np.abs(dt_bboxes-amodal_bboxes[j]),axis=1)
        final_index.append(np.argmin(dis))
    
    img = cv2.imread('../datasets/coco_amodal/val2014/'+img_info[i]['file_name']).astype('float')
    for j in final_index[::-1]:
        color = np.random.random((1, 3)).tolist()[0]
        mask = maskUtils.decode(dt[i]['regions'][j]['segmentation'])
        
        img = apply_mask(img, mask, color)

    cv2.imwrite(os.path.join(targetFolder,img_info[i]['file_name']),img)

FileNotFoundError: [Errno 2] No such file or directory: '../results/AmodalMask[39]/amodal-props-1-500.json'

In [24]:
################################################
##############   our results   #########
filenames = [item['file_name'] for item in img_info]
root = '/home/anpei/our_amodal_result/faillCase/'
json_list = os.listdir(root)
for item in json_list:
    if item.endswith('json'):
            
        f = open(root+item, "rb")
        files = pickle.load(f)
        
        amodal_bboxes = []
        i = filenames.index(item[:-5])
        for region in amodal.dataset['annotations'][i]['regions']:
            amodal_mask,vis_mask = amodal.getAnnMask2(region,img_info[i]['width'],img_info[i]['height'])
            bbox = mask_to_bbox(amodal_mask)
            amodal_bboxes.append(bbox)
            
        final_index = []
        dt_bboxes = files['rois']
        amodal_bboxes = np.array(amodal_bboxes)
        for j in range(amodal_bboxes.shape[0]):
            dis = np.sum(np.abs(dt_bboxes-amodal_bboxes[j]),axis=1)
            final_index.append(np.argmin(dis))
        
        if not os.path.exists(root+item[:-4]):
            os.mkdir(root+item[:-4])
        
        ind = 0
        img = cv2.imread('../datasets/coco_amodal/val2014/'+img_info[i]['file_name']).astype('float')
        for j in final_index:
            color = np.random.random((1, 3)).tolist()[0]
            mask = files['masks'][:,:,j]       
            img = apply_mask(img, mask, color)
   
            img = cv2.rectangle(img,(dt_bboxes[j][1],dt_bboxes[j][0]), (dt_bboxes[j][3],dt_bboxes[j][2]), np.array(color)*255, 2)
            name = os.path.join(root+item[:-4],'%02d.jpg' % ind )
            cv2.imwrite(name,mask*255)
            ind += 1

        cv2.imwrite(root+'our_'+img_info[i]['file_name'],img)

In [80]:
##############################################
##############   AmodalMask[39] results   #########
root = '../results/our_amodal_COCOA//'
img_info = amodal.dataset['images']
final_list = ['1533','3525','4154','8480','9514','9463','6341']
dx = 1000
for i in range(len(dt)):
    i += dx
    is_in_list = False
    for k in final_list:
        is_in_list += img_info[i]['file_name'].endswith(k+'.jpg')
    if not is_in_list:
        continue    
        
    dt_bboxes = []
    for region in dt[i-dx]['regions']:
        mask = maskUtils.decode(region['segmentation'])
        # TO DO get bounding box
        bbox = mask_to_bbox(mask)
        dt_bboxes.append(bbox)
        
    imgIds = dt[i-dx]['image_id']
    amodal_bboxes = []
    for region in amodal.dataset['annotations'][i]['regions']:
        amodal_mask,vis_mask = amodal.getAnnMask2(region,img_info[i]['width'],img_info[i]['height'])
        bbox = mask_to_bbox(amodal_mask)
        amodal_bboxes.append(bbox)
        
    dt_bboxes,amodal_bboxes = np.array(dt_bboxes),np.array(amodal_bboxes)
    
    if dt_bboxes.shape!=(1000,4):
        continue

    if not os.path.exists(root+img_info[i]['file_name'][:-4]):
        os.mkdir(root+img_info[i]['file_name'][:-4])
        

    final_index = []
    for j in range(amodal_bboxes.shape[0]):
        dis = np.sum(np.abs(dt_bboxes-amodal_bboxes[j]),axis=1)
        final_index.append(np.argmin(dis))
    
    img = cv2.imread('../datasets/coco_amodal/val2014/'+img_info[i]['file_name']).astype('float')
    
    ind = 0
    for j in final_index[::-1]:
        mask = maskUtils.decode(dt[i-dx]['regions'][j]['segmentation'])
        name = os.path.join(root+img_info[i]['file_name'][:-4],'%02d_' % ind + img_info[i]['file_name'])
        cv2.imwrite(name,mask*255)
        ind += 1
        
    name = os.path.join(root+img_info[i]['file_name'][:-4],img_info[i]['file_name'])
    cv2.imwrite(name,img)

8
8


In [ ]:
#################################################################
######################  D2S dataset  ############################

In [153]:
ann_file2 = '../datasets/D2S/annotations/COCO_amodal_val2014.json'
amodal = Amodal(ann_file2)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [190]:
##################### D2S DATASET ##################
img_info = amodal.dataset['images']
amodal.dataset['annotations']= sorted(amodal.dataset['annotations'], key=lambda ann: ann['image_id']) 
filenames = [item['file_name'] for item in img_info]
final_list = ['8302','4929','6022','8625','8312','4706','4811','8111']
root = '../results/our_amodal_D2SA/'
json_list = os.listdir(root)
for item in json_list:
    if item.endswith('json'):
        is_in_list = False
        amodal_bboxes = []
        i = filenames.index(item[:-5])
        for k in final_list:
            is_in_list += img_info[i]['file_name'].endswith(k+'.jpg')
        if not is_in_list:
            continue

        f = open(root+item, "rb")
        files = pickle.load(f)
        
        if not os.path.exists(root+img_info[i]['file_name'][:-4]):
            os.mkdir(root+img_info[i]['file_name'][:-4])
            
        for region in amodal.dataset['annotations'][i]['regions']:
            amodal_mask,vis_mask = amodal.getAnnMask2(region,img_info[i]['width'],img_info[i]['height'])
            bbox = mask_to_bbox(amodal_mask)
            amodal_bboxes.append(bbox)
            
            name = os.path.join(root+img_info[i]['file_name'][:-4],'gt_%02d_' %(len(amodal_bboxes)-1) + img_info[i]['file_name'])
            cv2.imwrite(name,amodal_mask*255)

            
        final_index = []
        dt_bboxes = files['rois']
        amodal_bboxes = np.array(amodal_bboxes)
        for j in range(amodal_bboxes.shape[0]):
            dis = np.sum(np.abs(dt_bboxes-amodal_bboxes[j]),axis=1)
            final_index.append(np.argmin(dis))
        
        masks = []

            
            
        img = cv2.imread('../datasets/D2S/val2014/'+img_info[i]['file_name']).astype('float')
        for j in final_index:
            mask = files['masks'][:,:,j]
            name = os.path.join(root+img_info[i]['file_name'][:-4],'our_%02d_' %j + img_info[i]['file_name'])
            
            cv2.imwrite(name,mask*255)

        name = os.path.join(root+img_info[i]['file_name'][:-4],img_info[i]['file_name'])
        cv2.imwrite(name,img)

10
3
3
2
8
2
3


In [ ]:
################################################################
#################### generate depth order result ###############

In [222]:
def IOU(mask_A,mask_B):
    return np.sum((mask_A*mask_B)>0)/np.sum((mask_A+mask_B)>0)

ann_file2 = '../datasets/coco_amodal/annotations/COCO_amodal_val2014.json'
amodal = Amodal(ann_file2)
img_info = amodal.dataset['images']
filenames = [item['file_name'] for item in img_info]
root = '/home/anpei/segementation/pytorch-mask-rcnn-coco-ppm/results/our_vis_only/'
save_root = '../datasets/coco_amodal/depth_order/vis_val2014/'
json_list = os.listdir(root)
for item in json_list:
    if item.endswith('json'):
        amodal_bboxes = []
        i = filenames.index(item[:-5])

        f = open(root+item, "rb")
        files = pickle.load(f)
        

        amodal_masks = []
        for region in amodal.dataset['annotations'][i]['regions']:
            amodal_mask,vis_mask = amodal.getAnnMask2(region,img_info[i]['width'],img_info[i]['height'])
            bbox = mask_to_bbox(amodal_mask)
            amodal_bboxes.append(bbox)
            amodal_masks.append(amodal_mask)
  
            
        final_index = []
        dt_bboxes = files['rois']
        amodal_bboxes = np.array(amodal_bboxes)
        for j in range(amodal_bboxes.shape[0]):
            dis = np.sum(np.abs(dt_bboxes-amodal_bboxes[j]),axis=1)
            final_index.append(np.argmin(dis))
        
        masks = []
        for j in final_index:
            masks.append(files['masks'][:,:,j])
        

        count = 0
        for i in range(len(amodal_masks)):
            for j in range(i+1,len(amodal_masks)):
                iou = IOU(amodal_masks[i],amodal_masks[j])

                if iou > 0.4 and final_index[i]!=final_index[j]:
                    res = np.stack((masks[i],masks[j]),axis=0)
                    name = os.path.join(save_root,item[:-9]+'_%d.depth'%count)
                    count += 1
                    np.save(name,res)
                    
                    #name = os.path.join(save_root,item[:-5])
                    #cv2.imwrite(name,np.concatenate((masks[i],masks[j]),axis=0)*255)
                    
        

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!


In [103]:
################## generate gt depth order
img_info = amodal.dataset['images']
save_root = '../datasets/coco_amodal/depth_order/val2014/'
for i in range(len(amodal.dataset['annotations'])):#
    item = img_info[i]['file_name']
    amodal_masks,vis_masks = []
    for region in amodal.dataset['annotations'][i]['regions']:
        amodal_mask,vis_mask = amodal.getAnnMask2(region,img_info[i]['width'],img_info[i]['height'])
        amodal_masks.append(amodal_mask)
        vis_masks.append(vis_mask)
        
    count = 0
    for i in range(len(amodal_masks)):
        for j in range(i+1,len(amodal_masks)):
            iou = IOU(amodal_masks[i],amodal_masks[j])

            if iou > 0.5 and iou < 0.9:
                res = np.stack((amodal_masks[i],amodal_masks[j]),axis=0)
                name = os.path.join(save_root,item[:-4]+'%d_%d.depth'%(i,j))
                count += 1
                np.save(name,res)

KeyboardInterrupt: 

In [203]:
################## R(X,Y) on GT
def R_xy(modal_masks_a,amodal_masks_b,vis_masks_a,vis_masks_b):
    intersetion = (modal_masks_a*amodal_masks_b)>0
    invis_a = modal_masks_a - vis_masks_a
    invis_b = amodal_masks_b - vis_masks_b
    return np.sum(intersetion*invis_a)<np.sum(intersetion*invis_b)
    
    
count,total = 0,0   
img_info = amodal.dataset['images']
for i in range(len(amodal.dataset['annotations'])):#
    item = img_info[i]['file_name']
    amodal_masks,vis_masks = [],[]
    for region in amodal.dataset['annotations'][i]['regions']:
        amodal_mask,vis_mask = amodal.getAnnMask2(region,img_info[i]['width'],img_info[i]['height'])
        if len(amodal_mask) and len(vis_mask):
            amodal_masks.append(amodal_mask)
            vis_masks.append(vis_mask)
        
    for i in range(len(amodal_masks)):
        for j in range(i+1,len(amodal_masks)):
            iou = IOU(amodal_masks[i],amodal_masks[j])

            if iou > 0.5:
                total += 1
                if R_xy(amodal_masks[i],amodal_masks[j],vis_masks[i],vis_masks[j]):
                    count += 1
                    
print(count/len(list_npy))

In [232]:
################## R(X,Y) on ours
def R_xy(modal_masks_a,amodal_masks_b,vis_masks_a,vis_masks_b):
    intersetion = (modal_masks_a*amodal_masks_b)>0
    invis_a = modal_masks_a - vis_masks_a
    invis_b = amodal_masks_b - vis_masks_b
    return np.sum(intersetion*invis_a)<np.sum(intersetion*invis_b)
    
root = '../datasets/coco_amodal/depth_order/val2014/'  
list_npy = os.listdir(root)
count,total = 0,0   
for item in list_npy:
    if item.endswith('npy'):
        mask = np.load(os.path.join(root,item))
        if R_xy(mask[0],mask[1],mask[2],mask[3]):
            count += 1
            
print(count/len(list_npy))

In [231]:
dir1 = '/home/anpei/segementation/pytorch-mask-rcnn-coco-ppm/datasets/coco_amodal/depth_order/amodal_val2014'
dir2 = '/home/anpei/segementation/pytorch-mask-rcnn-coco-ppm/datasets/coco_amodal/depth_order/vis_val2014'
save_root = '/home/anpei/segementation/pytorch-classification/datasets/coco_amodal/depth_order/val2014/'
npy_list = os.listdir(dir1)
for item in npy_list:
    if item.endswith('npy'):
        mask_1 = np.load(os.path.join(dir1,item))
        mask_2 = np.load(os.path.join(dir2,item))
        
        mask = np.concatenate((mask_1,mask_2),axis=0)
        #mask = mask[[0,2,1,3]]
        name = os.path.join(save_root,item)
        np.save(name,mask)